# **Setup**

In [ ]:
import sqlite3
import pandas as pd

# 1. Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')

# 2. Create a cursor object
c = conn.cursor()

# 3. Create a sample table: 'sales_data'
c.execute('''
CREATE TABLE sales_data (
    id INTEGER PRIMARY KEY,
    customer_name TEXT,
    category TEXT,
    amount REAL,
    transaction_date DATE
)
''')

# 4. Insert dummy data
data = [
    (1, 'Alice', 'Electronics', 120.00, '2023-01-01'),
    (2, 'Bob', 'Books', 25.00, '2023-01-02'),
    (3, 'Charlie', 'Electronics', 800.00, '2023-01-03'),
    (4, 'Alice', 'Books', 15.00, '2023-01-04'),
    (5, 'David', 'Home', 450.00, '2023-01-05'),
    (6, 'Bob', 'Electronics', 60.00, '2023-01-06'),
    (7, 'Eve', 'Home', 150.00, '2023-01-07'),
    (8, 'Alice', 'Electronics', 300.00, '2023-01-08'),
    (9, 'Charlie', 'Books', 40.00, '2023-01-09'),
    (10, 'David', 'Home', 200.00, '2023-01-10')
]

c.executemany('INSERT INTO sales_data VALUES (?,?,?,?,?)', data)
conn.commit()

# Helper function to run queries easily
def run_query(query):
    return pd.read_sql_query(query, conn)

print("✅ Database created and populated successfully!")

✅ Database created and populated successfully!


# **Module C | WHERE vs HAVING**

In [ ]:
query_where = """
SELECT
    customer_name,
    SUM(amount) as total_spend
FROM
    sales_data
WHERE
    category = 'Electronics'  -- Filters individual rows FIRST
GROUP BY
    customer_name
"""

display(run_query(query_where))

,customer_name,total_spend
0,Alice,420.0
1,Bob,60.0
2,Charlie,800.0


In [ ]:
query_having = """
SELECT
    customer_name,
    SUM(amount) as total_spend
FROM
    sales_data
GROUP BY
    customer_name
HAVING
    SUM(amount) > 500  -- Filters the GROUP result
"""

display(run_query(query_having))

,customer_name,total_spend
0,Charlie,840.0
1,David,650.0


# **Module D | Subqueries and CTEs**

## **Subqueries**
### What is a subquery?
- It is a query that runs inside another query
- It runs first
- Can return a single value or a full table

### **Subquery in SELECT**

In [ ]:
query_select_sub = """
SELECT
    customer_name,
    amount,
    -- This subquery runs once and pastes the result into every row
    (SELECT SUM(amount) FROM sales_data) as grand_total,
    ROUND(
        (amount / (SELECT SUM(amount) FROM sales_data)) * 100,
    2) as pct_of_total
FROM
    sales_data
ORDER BY
    amount DESC;
"""

display(run_query(query_select_sub))

,customer_name,amount,grand_total,pct_of_total
0,Charlie,800.0,2160.0,37.04
1,David,450.0,2160.0,20.83
2,Alice,300.0,2160.0,13.89
3,David,200.0,2160.0,9.26
4,Eve,150.0,2160.0,6.94
5,Alice,120.0,2160.0,5.56
6,Bob,60.0,2160.0,2.78
7,Charlie,40.0,2160.0,1.85
8,Bob,25.0,2160.0,1.16
9,Alice,15.0,2160.0,0.69


### **Subquery in WHERE**

In [ ]:
query = """
SELECT
    MAX(amount) as max_amount
FROM
    sales_data
"""

display(run_query(query))

,max_amount
0,800.0


In [ ]:
# Now we use that as a subquery!
query_subquery = """
SELECT
    customer_name,
    amount
FROM
    sales_data
WHERE
    amount = (
        SELECT MAX(amount)
        FROM sales_data
    )
"""

# Which customer ordered the MOST?
display(run_query(query_subquery))

,customer_name,amount
0,Charlie,800.0


In [ ]:
# Edit this query!
query_subquery = """
SELECT
    customer_name,
    amount
FROM
    sales_data
WHERE
    amount = (
        SELECT MAX(amount)
        FROM sales_data
    )
"""

# Which customer ordered the LEAST?
display(run_query(query_subquery))

,customer_name,amount
0,Charlie,800.0


### **Subquery in FROM**

In [ ]:
query = """
SELECT
    customer_name,
    SUM(amount) as customer_total
FROM
    sales_data
GROUP BY
    customer_name
"""

display(run_query(query))

,customer_name,customer_total
0,Alice,435.0
1,Bob,85.0
2,Charlie,840.0
3,David,650.0
4,Eve,150.0


In [ ]:
# Let's use that as a subquery!
query_derived = """
SELECT
    AVG(customer_total) as average_customer_lifetime_value
FROM
    (
        SELECT
            customer_name,
            SUM(amount) as customer_total
        FROM
            sales_data
        GROUP BY
            customer_name
    ) AS customer_spending_table;
"""

# What is the average customer lifetime value?
display(run_query(query_derived))

,average_customer_lifetime_value
0,432.0


## **CTEs**
(or Common Table Expressions)

In [ ]:
query = """
SELECT
    category,
    SUM(amount) as category_total,
    (SELECT SUM(amount) FROM sales_data) as grand_total
FROM
    sales_data
GROUP BY
    category
"""

display(run_query(query))

,category,category_total,grand_total
0,Books,80.0,2160.0
1,Electronics,1280.0,2160.0
2,Home,800.0,2160.0


In [ ]:
query_cte = """
WITH

category_agg AS (
    SELECT
        category,
        SUM(amount) as category_total,
        (SELECT SUM(amount) FROM sales_data) as grand_total
    FROM
        sales_data
    GROUP BY
        category
)

SELECT
    category,
    (category_total / grand_total) * 100 as percentage_share
FROM
    category_agg
"""

display(run_query(query_cte))

,category,percentage_share
0,Books,3.703704
1,Electronics,59.259259
2,Home,37.037037
